In [ ]:
import torch


In [ ]:
A = torch.Tensor([[0.3374, 0.6005, 0.1735], [3.3359, 0.0492, 1.8374], [2.9407, 0.5301, 2.2620]])
M = len(A)
N = len(A[0])
#rank = 2

U = torch.randn(M, rank, dtype=torch.float).uniform_()
V = torch.randn(N, rank, dtype=torch.float).uniform_()
print ()
print()
print(A)




tensor([[0.3374, 0.6005, 0.1735],
        [3.3359, 0.0492, 1.8374],
        [2.9407, 0.5301, 2.2620]])


In [ ]:
from typing import Tuple

def sgd_factorise(A, rank:int, num_epochs=1000, lr=0.01):
  for epoch in range(num_epochs):
    for row in range(len(A)):
      for col in range(len(A[row])):
        e = A[row][col] - U[row,:] @ (V[col,:]).T
        U[row,:] = U[row,:] + lr * e * V[col,:]
        V[col,:] = V[col,:] + lr * e * U[row,:]

  return U, V

In [ ]:
U, V = sgd_factorise(A, rank, num_epochs=1000, lr=0.01)
print(U)
print(V)

tensor([[ 0.6214, -0.1685],
        [ 0.2769,  1.6421],
        [ 0.9429,  1.2180]])
tensor([[ 0.8578,  1.8329],
        [ 0.8126, -0.1412],
        [ 0.8580,  1.0606]])


In [ ]:
rec_loss = torch.nn.functional.mse_loss(A, U @ V.T, size_average=None, reduce=None, reduction = 'sum')
rec_loss

tensor(0.1220)

In [ ]:
# rec_loss = torch.nn.functional.mse_loss(A, U @ torch.transpose(V, 0, 1), size_average=None, reduce=None, reduction = 'sum')
# rec_loss

In [ ]:
U, S, V = torch.svd(A)

print(U, S, V)

tensor([[-0.0801, -0.7448,  0.6625],
        [-0.7103,  0.5090,  0.4863],
        [-0.6994, -0.4316, -0.5697]]) tensor([5.3339, 0.6959, 0.3492]) tensor([[-0.8349,  0.2548,  0.4879],
        [-0.0851, -0.9355,  0.3430],
        [-0.5439, -0.2448, -0.8027]])


In [ ]:
U, S, V = torch.svd(A)
S[-1] = 0

rec_loss = torch.nn.functional.mse_loss(A, U @ torch.diag(S) @ V.T, size_average=None, reduce=None, reduction = 'sum')
rec_loss

tensor(0.1219)

In [ ]:
A = torch.Tensor([[0.3374, 0.6005, 0.1735], [ float('NaN'), 0.0492, 1.8374], [2.9407, float('NaN'), 2.2620]])
A

tensor([[0.3374, 0.6005, 0.1735],
        [   nan, 0.0492, 1.8374],
        [2.9407,    nan, 2.2620]])

In [ ]:
x = torch.ones_like(A)
y = torch.zeros_like(A)

M_nonan = torch.Tensor(A).nan_to_num(nan=0.0)
M = torch.where(M_nonan != 0, x, y)
M

tensor([[1., 1., 1.],
        [0., 1., 1.],
        [1., 0., 1.]])

In [ ]:
x = torch.ones_like(A)
y = torch.zeros_like(A)

M_nonan = torch.Tensor(A).nan_to_num(nan=0.0)
M = torch.where(M_nonan != 0, x, y)

def sgd_factorise(A, M, rank:int, num_epochs=1000, lr=0.01):
  for epoch in range(num_epochs):
    for row in range(len(A)):
      for col in range(len(A[row])):
        if M[row][col] != 0:
          e = A[row][col] - U[row,:] @ (V[col,:]).T
          U[row,:] = U[row,:] + lr * e * V[col,:]
          V[col,:] = V[col,:] + lr * e * U[row,:]

  return U, V

In [ ]:
U, V = sgd_factorise(A, M, rank, num_epochs=1000, lr=0.01)
print(U,V)

tensor([[-0.1867, -0.6302,  0.3805],
        [-1.5014,  0.2460, -0.0166],
        [-1.6102, -0.3714, -0.3698]]) tensor([[-1.8276e+00,  5.9897e-03, -2.3014e-04],
        [-1.5863e-01, -7.5072e-01,  2.5699e-01],
        [-1.2503e+00, -1.9365e-01, -4.7827e-01]])


In [ ]:
A_comp = U @ V.T
A_comp

tensor([[0.3374, 0.6005, 0.1735],
        [2.7454, 0.0492, 1.8374],
        [2.9407, 0.4392, 2.2620]])